In [1]:
import string
import pickle
import operator
import numpy as np
from nltk.corpus           import brown, stopwords
from sklearn.cluster       import KMeans
from sklearn.neighbors     import NearestNeighbors
from sklearn.decomposition import PCA


# Data prepping
# data_file 		  = open("striped_words.pickle",'w')
# striped_brwn 	  = [word.lower().strip(string.punctuation) for word in brown.words()]
# striped_stop_brwn = [word for word in striped_brwn if not(word in stopwords.words('english'))]


# pickle.dump(striped_stop_brwn,data_file)
# data_file.close()
# print(len(striped_brwn))
# print(len(striped_stop_brwn))

# Data Reading
data_file = open("striped_words.pickle",'r')
words 	  = pickle.load(data_file)

In [2]:
word_count = dict()
for word in words:
	if not(word in word_count):
		word_count[word]=1
	else:
		word_count[word]+=1

sorted_word_count = sorted(word_count.items(), key=operator.itemgetter(1))
sorted_word_count.reverse()
sorted_word_count = sorted_word_count[1:]

In [3]:
five_words = sorted_word_count[0:5000]
one_words  = sorted_word_count[0:1000]
all_five_words = [datum[0] for datum in five_words]
all_one_words  = [datum[0] for datum in one_words]
joint_one_words= zip(all_one_words,[0.0]*len(all_one_words))



In [4]:
nwc 	= np.zeros((5000,1000))
pc_dict = dict(joint_one_words)
# for word in all_five_words:
# 	nwc[word] = dict(joint_one_words)


for i in range(len(words)):          # Iterate through all text
    if(words[i] in all_five_words):  # If word within the top 5000 words
        parent_idx = all_five_words.index(words[i])
        # Iterate though w1, w2, w3, w4
        # w1
        # print(words[i-1])
        if((i-1)>=0 and words[i-1] in all_one_words):
            child_idx = all_one_words.index(words[i-1])
            nwc[parent_idx,child_idx]+=1

        # w2
        if((i-2)>=0 and words[i-2] in all_one_words):
            child_idx = all_one_words.index(words[i-2])
            nwc[parent_idx,child_idx]+=1

        # w3
        if((i+1)<len(one_words) and words[i+1] in all_one_words):
            child_idx = all_one_words.index(words[i+1])
            nwc[parent_idx,child_idx]+=1

        # w4
        if((i+2)<len(one_words) and words[i+2] in all_one_words):
            child_idx = all_one_words.index(words[i+2])
            nwc[parent_idx,child_idx]+=1


In [5]:
pwc=np.copy(nwc)
for i in range(len(all_five_words)):
    total = sum(pwc[i,:])
    if(total ==0):
        continue
    pwc[i,:]=pwc[i,:]/float(total)

In [6]:
total_c = sum([datum[1] for datum in one_words])
pc = np.zeros(1000)
for i in range(1000):
    pc[i] = one_words[i][1]/float(total_c)

In [7]:
for data in pc:
    if(data==0):
        print("yes")

In [8]:
sigma_w=np.zeros((5000,1000))
for i in range(5000):
    for j in range(1000):
        if(pwc[i,j]/pc[j]>0 and np.log(pwc[i,j]/pc[j])>0):  
            sigma_w[i,j]=np.log(pwc[i,j]/pc[j])
print(all_one_words)

[u'one', u'would', u'said', u'new', u'could', u'time', u'two', u'may', u'first', u'like', u'man', u'even', u'made', u'also', u'many', u'must', u'af', u'back', u'years', u'much', u'way', u'well', u'people', u'mr', u'little', u'state', u'good', u'make', u'world', u'still', u'see', u'men', u'work', u'long', u'get', u'life', u'never', u'day', u'another', u'know', u'last', u'us', u'might', u'great', u'old', u'year', u'come', u'since', u'go', u'came', u'right', u'states', u'used', u'three', u'take', u'house', u'use', u'without', u'place', u'american', u'around', u'however', u'home', u'1', u'small', u'found', u'mrs', u'thought', u'went', u'say', u'part', u'general', u'high', u'upon', u'school', u'every', u'united', u'got', u'left', u'number', u'course', u'2', u'war', u'always', u'away', u'something', u'fact', u'water', u'though', u'public', u'less', u'put', u'think', u'almost', u'hand', u'enough', u'far', u'took', u'head', u'yet', u'government', u'system', u'better', u'set', u'told', u'nothin

In [9]:
test_words = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind','africa',
              'chicago', 'revolution','september','chemical','detergent','dictionary',
              'storm', 'worship']
def cos_sim(x, y, fet_len, embed):
    numer= np.dot(embed[x[0],x[2]],embed[y[0],y[1]])
    denom= np.linalg.norm(embed[x[0],x[2]])*np.linalg.norm(embed[x[0],x[2]])
    return(1-(numer/denom))

    

In [10]:
pca = PCA(n_components=100)
pca.fit(sigma_w)
reduced_sigma_w = pca.fit_transform(sigma_w)

In [11]:
nearest_mod         = NearestNeighbors(n_neighbors=2, algorithm='auto', metric="cosine").fit(sigma_w)
reduced_nearest_mod = NearestNeighbors(n_neighbors=2, algorithm='auto', metric="cosine").fit(reduced_sigma_w)
for word in test_words:
    idx = all_five_words.index(word)
    dist_a, nrst_idx_a = nearest_mod.kneighbors(sigma_w[idx,:].reshape((1,1000)))
    dist_b, nrst_idx_b = reduced_nearest_mod.kneighbors(reduced_sigma_w[idx,:].reshape((1,100)))
    print(word, all_five_words[nrst_idx_a[0,1]], all_five_words[nrst_idx_b[0,1]])

('communism', u'utopian', u'utopian')
('autumn', u'whip', u'summer')
('cigarette', u'andy', u'andy')
('pulmonary', u'artery', u'artery')
('mankind', u'christ', u'christ')
('africa', u'asia', u'germany')
('chicago', u'washington', u'board')
('revolution', u'violent', u'civil')
('september', u'december', u'november')
('chemical', u'kind', u'characteristics')
('detergent', u'liquid', u'foam')
('dictionary', u'text', u'stored')
('storm', u'saturday', u'saturday')
('worship', u'prize', u'shared')


In [12]:
cluster_a = KMeans(n_clusters=100, random_state=0, precompute_distances=True).fit(sigma_w)
cluster_b = KMeans(n_clusters=100, random_state=0, precompute_distances=True).fit(reduced_sigma_w)
print(cluster_a.cluster_centers_)

[[ 0.          0.          1.04244578 ...,  0.          2.55611225  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.23386624  0.         ...,  0.          0.          0.        ]
 [ 1.26522855  1.17213588  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [24]:
centroid_a = cluster_a.cluster_centers_
centroid_b = cluster_b.cluster_centers_
for i in range(0,100,2):
    dist_a, nrst_idx_a = nearest_mod.kneighbors(centroid_a[i,:].reshape((1,1000)))
    dist_a, nrst_idx_b = nearest_mod.kneighbors(centroid_a[i+1,:].reshape((1,1000)))
    print(all_five_words[nrst_idx_a[0,1]]+"       \t"+all_five_words[nrst_idx_b[0,1]])

attacks       	grabbed
island       	warmth
owen       	three
system       	anode
travel       	data
companies       	test
truly       	loud
charged       	house
must       	review
logical       	information
passages       	struggle
skirt       	tax
reading       	miles
medical       	training
find       	times
podger       	distinct
world       	could
dictionary       	2
point       	f
railroad       	jesus
per       	house
completion       	see
acceptance       	cent
study       	realism
what's       	around
new       	dominant
sake       	car
11       	highly
local       	servant
around       	toward
problem       	effects
research       	june
distinguish       	cope
market       	experience
became       	toward
types       	account
stomach       	east
equipment       	million
education       	social
values       	high
affairs       	interest
tax       	law
new       	appeal
1960       	days
parties       	russians
medical       	well
beef       	many
question       	secretary
blues

In [27]:
for i in range(0,100,2):
    dist_a, nrst_idx_a = reduced_nearest_mod.kneighbors(centroid_b[i,:].reshape((1,100)))
    dist_a, nrst_idx_b = reduced_nearest_mod.kneighbors(centroid_b[i+1,:].reshape((1,100)))
    print(all_five_words[nrst_idx_a[0,1]]+"       \t"+all_five_words[nrst_idx_b[0,1]])

shoulders       	couple
protein       	discrimination
surface       	received
simply       	excitement
decision       	go
computed       	feb
stuck       	miles
canada       	1950
immortality       	experience
court       	possible
value       	places
yeah       	vacuum
yankees       	15
hoag       	took
edition       	ambassador
costs       	around
education       	meaningful
much       	evident
4       	even
version       	enforced
amen       	nature
i.e       	committee
building       	carolina
institutions       	protection
pleasure       	november
tell       	due
lb       	he's
situations       	sad
effect       	companies
techniques       	jazz
per       	cost
development       	take
production       	mistake
dark       	require
state       	scene
front       	areas
f       	times
america       	purposes
porch       	father
calif       	girl
program       	evidence
plug       	issued
face       	alert
donald       	1961
policy       	two
side       	distinguish
days       	hair
b

In [33]:
cluster_a_size = np.zeros(100,dtype='int32')
cluster_b_size = np.zeros(100,dtype='int32')
for i in range(5000):
    idx_a = cluster_a.predict(sigma_w[i,:].reshape((1,1000)))
    idx_b = cluster_b.predict(reduced_sigma_w[i,:].reshape((1,100)))
    cluster_a_size[idx_a]+=1
    cluster_b_size[idx_b]+=1
print(cluster_a_size)
print(cluster_b_size)

[   2    1    2    1    1   29    3   10    4    4    1   14    1   57   97
    1  254    4    4    1    1    3    1    1   78    3    1    9  433  115
  165    8    1  244   22    5    1   11    5    1    1    1    1   50    1
   50    1    1    1  433    1    1    1    1    9   16    1    1   58    1
    1    1    1    1  123    1   72    1   47    1   32    1    1  116    1
    2   72  174    1    1    1  147    1    1    1    1    1   10    1    1
    1    1    9 1897    1    1   43    1    1    1]
[ 21  16  73  76   8  61  16  96  64  20  65  85  86  11  89  92  87  59
  35  16  52   6 266 120  49  25 117  39 101  90  28  16  65 102  56  48
   8 110  43  81  91   3  85  15   5  70  43  55  51  29  49  50  67  75
   1  40  34  59  29  22  14   2  12  18  24 121  18  36  19  51  35   3
  12  92  31  66  75  12  96  29   5   2 143 102   2  70 101   6  12  26
  22  39  20  58  82  12  30   6  78  47]


In [34]:
sorted_a_size = cluster_a_size.argsort()
sorted_b_size = cluster_b_size.argsort()

In [81]:
for i in range(5000):
    idx_a = cluster_a.predict(sigma_w[i,:].reshape((1,1000)) )
#     idx_b = cluster_b.predict(reduced_sigma_w[i,:].reshape((1,100)))
    if(idx_a==sorted_a_size[65]):
        print(all_five_words[i])

obviously
railroad
questionnaire
reorganization
merger


In [68]:
for i in range(5000):
#     idx_a = cluster_a.predict(sigma_w[i,:].reshape((1,1000)) )
    idx_b = cluster_b.predict(reduced_sigma_w[i,:].reshape((1,100)))
    if(idx_b==sorted_b_size[14]):
        print(all_five_words[i])

world
city
group
church
family
body
society
community
party
county
nation
population
